# Final Project - Beat the Bookies

## New notebook - exploring hyper-parameters

In [6]:
import pandas as pd 
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

### Import the data

In [5]:
# import the data
path = '/Users/georgebrockman/code/georgebrockman/beatthebookies/beatthebookies/data/premiertotals.csv'
data = pd.read_csv(path)
data.head()

,season,date,home_team,away_team,home_team_goal,away_team_goal,FTR,HTHG,HTAG,HTR,Referee,home_shots,away_shots,home_shots_ot,away_shots_ot,home_fouls,away_fouls,home_corn,away_corn,home_yel,away_yel,home_red,away_red,WHH,WHD,WHA,H_ATT,H_MID,H_DEF,H_OVR,A_ATT,A_MID,A_DEF,A_OVR,home_w,draw,away_w,winning_odds,home_t_home_goals,home_t_home_goals_against,home_t_home_shots,home_t_home_shots_ot,home_t_home_shots_against,home_t_home_shots_ot_against,home_t_home_fouls,home_t_home_fouls_against,home_t_home_corn,home_t_home_corn_against,home_t_home_yel,home_t_home_yel_against,home_t_home_red,home_t_home_red_against,home_t_home_wins,home_t_home_losses,home_t_home_draws,away_t_away_goals,away_t_away_goals_against,away_t_away_shots,away_t_away_shots_ot,away_t_away_shots_against,away_t_away_shots_ot_against,away_t_away_fouls,away_t_away_fouls_against,away_t_away_corn,away_t_away_corn_against,away_t_away_yel,away_t_away_yel_against,away_t_away_red,away_t_away_red_against,away_t_away_wins,away_t_away_losses,away_t_away_draws,home_t_prev_home_matches,away_t_prev_away_matches,stage,home_t_total_goals,home_t_total_goals_against,home_t_total_shots,home_t_total_shots_against,home_t_total_shots_ot,home_t_total_shots_ot_against,home_t_total_fouls,home_t_total_fouls_against,home_t_total_corn,home_t_total_corn_against,home_t_total_yel,home_t_total_yel_against,home_t_total_red,home_t_total_red_against,home_t_total_wins,home_t_total_losses,home_t_total_draws,away_t_total_goals,away_t_total_goals_against,away_t_total_shots,away_t_total_shots_against,away_t_total_shots_ot,away_t_total_shots_ot_against,away_t_total_fouls,away_t_total_fouls_against,away_t_total_corn,away_t_total_corn_against,away_t_total_yel,away_t_total_yel_against,away_t_total_red,away_t_total_red_against,away_t_total_wins,away_t_total_losses,away_t_total_draws,under_win
0,2008/2009,2008-08-16,Arsenal,West Brom,1.0,0.0,H,1.0,0.0,H,H Webb,24.0,5.0,14.0,4.0,11.0,8.0,7.0,5.0,0.0,0.0,0.0,0.0,1.17,5.5,12.00,85,83,83,81,68,71,69,71,1,0,0,1.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
1,2008/2009,2008-08-16,Bolton,Stoke,3.0,1.0,H,3.0,0.0,H,C Foy,14.0,8.0,8.0,2.0,13.0,12.0,4.0,3.0,1.0,2.0,0.0,0.0,1.80,3.1,4.00,75,75,72,74,75,68,67,70,1,0,0,1.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2,2008/2009,2008-08-16,Everton,Blackburn,2.0,3.0,A,1.0,1.0,D,A Marriner,10.0,15.0,5.0,11.0,11.0,9.0,3.0,5.0,2.0,2.0,0.0,0.0,1.95,3.1,3.50,82,75,78,78,78,76,76,76,0,0,1,3.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
3,2008/2009,2008-08-16,Hull,Fulham,2.0,1.0,H,1.0,1.0,D,P Walton,11.0,12.0,6.0,6.0,10.0,9.0,5.0,6.0,3.0,0.0,0.0,0.0,2.50,2.9,2.62,70,68,68,71,80,74,72,74,1,0,0,2.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
4,2008/2009,2008-08-16,Middlesbrough,Tottenham,2.0,1.0,H,0.0,0.0,D,M Atkinson,14.0,8.0,10.0,5.0,11.0,12.0,7.0,9.0,1.0,2.0,0.0,0.0,2.90,3.2,2.15,80,75,73,74,80,80,79,79,1,0,0,2.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0


### Import the encoders

In [7]:
class FifaDifferentials(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.H_ATT = "H_ATT"
        self.H_MID = "H_MID"
        self.H_DEF = "H_DEF"
        self.H_OVR = "H_OVR"

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        assert isinstance(X, pd.DataFrame)
        # home team differentials
        X['H_ATT_D'] = X['H_ATT'] - X['A_ATT']
        X['H_MID_D'] = X['H_MID'] - X['A_MID']
        X['H_DEF_D'] = X['H_DEF'] - X['A_DEF']
        X['H_OVR_D'] = X['H_OVR'] - X['A_OVR']
        X['H_ATT_A'] = X['H_ATT'] - X['A_DEF']
        X['H_DEF_A'] = X['H_DEF'] - X['A_ATT']

        return X[['H_ATT_D', 'H_MID_D', 'H_DEF_D', 'H_OVR_D', 'H_ATT_A', 'H_DEF_A']]

In [8]:
class WeeklyGoalAverages(BaseEstimator, TransformerMixin):

    def __init__(self):
        # self.home_t_total_goals = 'home_t_home_goals'
        # self.away_t_total_goals = 'away_t_away_goals'
        self.home_t_total_goals = 'home_t_total_goals'
        self.away_t_total_goals = 'away_t_total_goals'
        # self.prev_home_matches = 'home_t_prev_home_matches'
        # self.prev_away_matches = 'away_t_prev_away_matches'

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        assert isinstance(X, pd.DataFrame)
        # X['home_t_average_home_goals'] = X['home_t_home_goals'] / X['home_t_prev_home_matches']
        # X['away_t_average_away_goals'] = X['away_t_away_goals'] / X['away_t_prev_away_matches']
        X['home_t_average_goals'] = X['home_t_total_goals'] / (X['stage'] - 1)
        X['away_t_average_goals'] = X['away_t_total_goals'] / (X['stage'] - 1)
        # dividing by zero returns infinite and NaN values
        X.replace([np.inf, np.nan], 0, inplace=True)
        X['home_t_average_goals_diff'] = X['home_t_average_goals'] - X['away_t_average_goals']

        # return X[['home_t_average_home_goals', 'home_t_average_goals', 'away_t_average_goals', 'away_t_average_away_goals']]
        return X[['home_t_average_goals_diff']]

In [9]:
class WeeklyGoalAgAverages(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        assert isinstance(X, pd.DataFrame)
        X['home_t_average_goals_against'] = X['home_t_total_goals_against'] / (X['stage'] - 1)
        X['away_t_average_goals_against'] = X['away_t_total_goals_against'] / (X['stage'] - 1)
        # dividing by zero returns infinite and NaN values
        X.replace([np.inf, np.nan], 0, inplace=True)
        X['home_t_average_goals_ag_diff'] = X['home_t_average_goals_against'] - X['away_t_average_goals_against']

        return X[['home_t_average_goals_ag_diff']]

In [10]:
class WinPctDifferentials(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        assert isinstance(X, pd.DataFrame)
        X['home_t_win_pct'] = X['home_t_total_wins'] / (X['stage'] - 1)
        X['away_t_win_pct'] = X['away_t_total_wins'] / (X['stage'] - 1)
        # dividing by zero returns infinite and NaN values
        X.replace([np.inf, np.nan], 0, inplace=True)
        X['home_t_win_pct_diff'] = X['home_t_win_pct'] - X['away_t_win_pct']

        return X[['home_t_win_pct_diff']]

In [11]:
class HomeAdv(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        assert isinstance(X, pd.DataFrame)
        X['home_t_avg_h_g'] = X['home_t_home_goals'] / (X['home_t_prev_home_matches'])
        X['home_t_avg_h_g_a'] = X['home_t_home_goals_against'] / (X['home_t_prev_home_matches'])
        X['away_t_avg_a_g_a'] = X['away_t_away_goals_against'] / (X['away_t_prev_away_matches'])
        X['away_t_avg_a_g'] = X['away_t_away_goals'] / (X['away_t_prev_away_matches'])
        # dividing by zero returns infinite and NaN values
        X.replace([np.inf, np.nan], 0, inplace=True)

        return X[['home_t_avg_h_g', 'home_t_avg_h_g_a', 'away_t_avg_a_g_a', 'away_t_avg_a_g']]

In [12]:
class ShotOTPct(BaseEstimator, TransformerMixin):
    def __init__(self):
          pass


    def fit(self, X, y=None):
          return self

    def transform(self, X, y=None):
        assert isinstance(X, pd.DataFrame)
        X['home_t_shototpct'] = X['home_t_total_shots_ot'] / X['home_t_total_shots']
        X['away_t_shototpct'] = X['away_t_total_shots_ot'] / X['away_t_total_shots']
        # dividing by zero returns infinite and NaN values
        X.replace([np.inf, np.nan], 0, inplace=True)
        X['home_t_shototpct_diff'] = X['home_t_shototpct'] - X['away_t_shototpct']

        return X[['home_t_shototpct_diff']]

### Build the pipeline

In [94]:
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.naive_bayes import GaussianNB

from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss

In [15]:
pipe_fifadiff = make_pipeline(FifaDifferentials(), RobustScaler())
pipe_winpct = make_pipeline(WinPctDifferentials(), StandardScaler())
pipe_avggoal = make_pipeline(WeeklyGoalAverages(), StandardScaler())
pipe_shototpct = make_pipeline(ShotOTPct())
pipe_avggoal_ag = make_pipeline(WeeklyGoalAgAverages(), StandardScaler())
pipe_home_adv = make_pipeline(HomeAdv(), StandardScaler())

In [16]:
feateng_blocks = [('fifadiff', pipe_fifadiff, ['H_ATT', 'A_ATT', 'H_MID', 'A_MID', 'H_DEF', 'A_DEF', 'H_OVR', 'A_OVR']),
                          ('windiff', pipe_winpct, ['home_t_total_wins','away_t_total_wins', 'stage']),
                          ('goaldiff', pipe_avggoal, ['home_t_total_goals','away_t_total_goals', 'stage']),
                          ('homeadv', pipe_home_adv, ['home_t_home_goals','home_t_home_goals_against','away_t_away_goals','away_t_away_goals_against',
                                                      'home_t_prev_home_matches', 'away_t_prev_away_matches']),
                          ('shototpct', pipe_shototpct, ['home_t_total_shots', 'home_t_total_shots_ot', 'away_t_total_shots', 'away_t_total_shots_ot']),
                          ('goalagdiff', pipe_avggoal_ag, ['home_t_total_goals_against','away_t_total_goals_against', 'stage'])
                         ]
features_encoder = ColumnTransformer(feateng_blocks, n_jobs=None, remainder="drop")

In [95]:
model = Pipeline(steps=[('features', features_encoder),('svm', LinearSVC(class_weight='balanced'))])

In [96]:
model

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('fifadiff',
                                                  Pipeline(steps=[('fifadifferentials',
                                                                   FifaDifferentials()),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['H_ATT', 'A_ATT', 'H_MID',
                                                   'A_MID', 'H_DEF', 'A_DEF',
                                                   'H_OVR', 'A_OVR']),
                                                 ('windiff',
                                                  Pipeline(steps=[('winpctdifferentials',
                                                                   WinPctDifferentials()),
                                                                  ('standardscaler',
         

### Train test split 

In [97]:
data.columns.values

array(['season', 'date', 'home_team', 'away_team', 'home_team_goal',
       'away_team_goal', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee',
       'home_shots', 'away_shots', 'home_shots_ot', 'away_shots_ot',
       'home_fouls', 'away_fouls', 'home_corn', 'away_corn', 'home_yel',
       'away_yel', 'home_red', 'away_red', 'WHH', 'WHD', 'WHA', 'H_ATT',
       'H_MID', 'H_DEF', 'H_OVR', 'A_ATT', 'A_MID', 'A_DEF', 'A_OVR',
       'home_w', 'draw', 'away_w', 'winning_odds', 'home_t_home_goals',
       'home_t_home_goals_against', 'home_t_home_shots',
       'home_t_home_shots_ot', 'home_t_home_shots_against',
       'home_t_home_shots_ot_against', 'home_t_home_fouls',
       'home_t_home_fouls_against', 'home_t_home_corn',
       'home_t_home_corn_against', 'home_t_home_yel',
       'home_t_home_yel_against', 'home_t_home_red',
       'home_t_home_red_against', 'home_t_home_wins',
       'home_t_home_losses', 'home_t_home_draws', 'away_t_away_goals',
       'away_t_away_goals_against', 'away_t

In [98]:
data1 = data[data['season'] != '2019/2020']

In [99]:
features = ['H_ATT',
       'H_MID', 'H_DEF', 'H_OVR', 'A_ATT', 'A_MID', 'A_DEF', 'A_OVR',
       'home_w', 'draw', 'away_w', 'winning_odds', 'home_t_home_goals',
       'home_t_home_goals_against', 'home_t_home_shots',
       'home_t_home_shots_ot', 'home_t_home_shots_against',
       'home_t_home_shots_ot_against', 'home_t_home_fouls',
       'home_t_home_fouls_against', 'home_t_home_corn',
       'home_t_home_corn_against', 'home_t_home_yel',
       'home_t_home_yel_against', 'home_t_home_red',
       'home_t_home_red_against', 'home_t_home_wins',
       'home_t_home_losses', 'home_t_home_draws', 'away_t_away_goals',
       'away_t_away_goals_against', 'away_t_away_shots',
       'away_t_away_shots_ot', 'away_t_away_shots_against',
       'away_t_away_shots_ot_against', 'away_t_away_fouls',
       'away_t_away_fouls_against', 'away_t_away_corn',
       'away_t_away_corn_against', 'away_t_away_yel',
       'away_t_away_yel_against', 'away_t_away_red',
       'away_t_away_red_against', 'away_t_away_wins',
       'away_t_away_losses', 'away_t_away_draws',
       'home_t_prev_home_matches', 'away_t_prev_away_matches', 'stage',
       'home_t_total_goals', 'home_t_total_goals_against',
       'home_t_total_shots', 'home_t_total_shots_against',
       'home_t_total_shots_ot', 'home_t_total_shots_ot_against',
       'home_t_total_fouls', 'home_t_total_fouls_against',
       'home_t_total_corn', 'home_t_total_corn_against',
       'home_t_total_yel', 'home_t_total_yel_against', 'home_t_total_red',
       'home_t_total_red_against', 'home_t_total_wins',
       'home_t_total_losses', 'home_t_total_draws', 'away_t_total_goals',
       'away_t_total_goals_against', 'away_t_total_shots',
       'away_t_total_shots_against', 'away_t_total_shots_ot',
       'away_t_total_shots_ot_against', 'away_t_total_fouls',
       'away_t_total_fouls_against', 'away_t_total_corn',
       'away_t_total_corn_against', 'away_t_total_yel',
       'away_t_total_yel_against', 'away_t_total_red',
       'away_t_total_red_against', 'away_t_total_wins',
       'away_t_total_losses', 'away_t_total_draws']

In [100]:
# import library from sklearn
from sklearn.model_selection import train_test_split
# test size is best around 30% of the data
X_train,X_test,y_train,y_test = train_test_split(data1[features],
                                     data1['under_win'],test_size=0.3,random_state=3) # arbitrary 

In [101]:
model.fit(X_train, y_train)

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('fifadiff',
                                                  Pipeline(steps=[('fifadifferentials',
                                                                   FifaDifferentials()),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['H_ATT', 'A_ATT', 'H_MID',
                                                   'A_MID', 'H_DEF', 'A_DEF',
                                                   'H_OVR', 'A_OVR']),
                                                 ('windiff',
                                                  Pipeline(steps=[('winpctdifferentials',
                                                                   WinPctDifferentials()),
                                                                  ('standardscaler',
         

In [110]:
model.score(X_train, y_train)

0.5598086124401914

In [103]:
data2 = data[data['season'] == '2019/2020']

In [115]:
predicts = model.predict(data2[features])

In [116]:
np.unique(predicts, return_counts=True)

(array([0, 1]), array([199, 181]))

In [117]:
actuals = data2['under_win']

In [118]:
actuals.sum()

89

In [119]:
from sklearn.metrics import classification_report

In [120]:
print(classification_report(actuals, predicts, target_names=['under_no_win', 'under_win']))

              precision    recall  f1-score   support

under_no_win       0.79      0.54      0.64       291
   under_win       0.27      0.54      0.36        89

    accuracy                           0.54       380
   macro avg       0.53      0.54      0.50       380
weighted avg       0.67      0.54      0.58       380



### Complete NN

In [133]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.layers import Embedding, Conv1D, Dense, Flatten, SimpleRNN, Conv2D, MaxPooling1D,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from keras.layers import BatchNormalization
from sklearn.model_selection import cross_val_score

In [132]:
data1.shape # after pipeline the shame is (4180, 14)

(4180, 110)

In [173]:
model = Sequential()
# model.add(BatchNormalization())
model.add(Dense(32, input_shape=(14,), activation='relu'))
model.add(Dense(10, activation='tanh'))
### Classification with 2 classes : 
model.add(Dense(1, activation='sigmoid'))

In [174]:
model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 32)                480       
_________________________________________________________________
dense_65 (Dense)             (None, 10)                330       
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 11        
Total params: 821
Trainable params: 821
Non-trainable params: 0
_________________________________________________________________


In [175]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [176]:
pipe = Pipeline(steps=[('features', features_encoder),('model', model)])

In [180]:
pipe.fit(X_train, y_train)

92/92 [==============================] - 0s 607us/step - loss: 0.5141 - accuracy: 0.7700


Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('fifadiff',
                                                  Pipeline(steps=[('fifadifferentials',
                                                                   FifaDifferentials()),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['H_ATT', 'A_ATT', 'H_MID',
                                                   'A_MID', 'H_DEF', 'A_DEF',
                                                   'H_OVR', 'A_OVR']),
                                                 ('windiff',
                                                  Pipeline(steps=[('winpctdifferentials',
                                                                   WinPctDifferentials()),
                                                                  ('standardscaler',
         

In [191]:
pipe_predicts = pipe.predict(data2[features])

In [192]:
pipe_predicts

array([[0.13271958],
       [0.27508837],
       [0.14735332],
       [0.32049415],
       [0.22459611],
       [0.29154453],
       [0.27002925],
       [0.271859  ],
       [0.30619818],
       [0.31001714],
       [0.09329662],
       [0.31422973],
       [0.30342376],
       [0.15228003],
       [0.28343338],
       [0.30006766],
       [0.14108887],
       [0.33230835],
       [0.2365219 ],
       [0.30186003],
       [0.2436539 ],
       [0.15380245],
       [0.16699153],
       [0.30934292],
       [0.18292984],
       [0.11157587],
       [0.06897023],
       [0.16903284],
       [0.05454072],
       [0.21309173],
       [0.28185642],
       [0.12127611],
       [0.17829755],
       [0.21861067],
       [0.10511494],
       [0.27646413],
       [0.1351054 ],
       [0.28667718],
       [0.31561506],
       [0.27972567],
       [0.1782799 ],
       [0.24893537],
       [0.12409508],
       [0.30747175],
       [0.1897358 ],
       [0.35323638],
       [0.06109235],
       [0.272

In [197]:
convert = pd.DataFrame.from_records(pipe_predicts, columns=['predict'])

In [198]:
convert

,predict
0,0.132720
1,0.275088
2,0.147353
3,0.320494
4,0.224596
5,0.291545
6,0.270029
7,0.271859
8,0.306198
9,0.310017


In [207]:
def predict_threshold(x):
            if x['predict'] >=  0.5:
                return 1
            return 0

In [208]:
convert['predict'] = convert.apply(lambda x: predict_threshold(x), axis=1)

In [209]:
convert

,predict
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0
